In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import sys
pt_models = "../input/pretrained-models/pretrained-models.pytorch-master/"
sys.path.insert(0,pt_models)
import pretrainedmodels

import glob
import torch
import albumentations
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
from tqdm import tqdm
from PIL import Image
import joblib
import torch.nn as nn
from torch.nn import functional as F

In [2]:
TEST_BATCH_SIZE = 32
MODEL_MEAN=(0.485,0.465,0.406)
MODEL_STD=(0.229,0.224,0.225)
IMG_HEIGHT=137
IMG_WIDTH=236
DEVICE="cuda"

In [3]:
class ResNet34(nn.Module):
    def __init__(self ,pretrained):
        super(ResNet34,self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained="imagenet")
        else:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained=None)

        # To replace the last layer of the model with these
        self.layer0 = nn.Linear(512,168) # 168 grapheme_root
        self.layer1 = nn.Linear(512,11) # 11 vowel_diacritic
        self.layer2 = nn.Linear(512,7) # 7 consonant_diacritic

    def forward(self,x):
        batch_size ,_,_,_ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size, -1)
        layer0 = self.layer0(x)
        layer1 = self.layer1(x)
        layer2 = self.layer2(x)
        return layer0, layer1, layer2 # grapheme_root, vowel_diacritic, consonant_diacritic



In [5]:
class BengaliDatasetTest:
    def __init__(self,df, img_height,img_width, mean, std):
        self.image_ids = df.image_id.values
        self.img_arr =df.iloc[:, 1:].values
        
        self.aug = albumentations.Compose([
            albumentations.Resize(img_height,img_width,always_apply=True),
            albumentations.Normalize(mean,std, always_apply=True)
        ])


    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self,item):
        image = self.img_arr[item,:]
        img_id = self.image_ids[item]
        image = image.reshape(137,236).astype(float)
        image = Image.fromarray(image).convert("RGB") #even though data is in black and white, we are using RGB as most pretrained network takes input in RGB
        image = self.aug(image=np.array(image))["image"]
        image = np.transpose(image,(2,0,1)).astype(np.float32) #convert to match torchvision input size
        return {
            'image': torch.tensor(image,dtype=torch.float),
            'image_id':img_id
        }

In [7]:
model = ResNet34(pretrained=False)
model.load_state_dict(torch("../input/bengali_models/resnet34_fold0.bin")) # load dataparallel if trained the model using multiple gpus
model.eval

TypeError: 'module' object is not callable

In [8]:
predictions=[]
for file_idx in range(4):
    df = pd.read_parquet(f"../input/bengaliai-cv19/test_image_data_{file_idx}.parquet")
    dataset = BengaliDatasetTest(df=df,img_height=IMG_HEIGHT,img_width=IMG_WIDTH,
                                mean=MODEL_MEAN,std=MODEL_STD)
    data_loader = torch.utils.data.DataLoader(dataset,batch_size=TEST_BATCH_SIZE,shuffle=False,num_workers=4)
    for bi , d in enumerate(dataset):
        image = d["image"]
        img_id= d["image_id"]
        image= image.to(DEVICE,dtype=torch.float)
        g,v,c=model(image)
        g = np.argmax(g,axis=1) 
        v = np.argmax(v,axis=1)
        c = np.argmax(c,axis=1)
        for i,im_id in enumerate(img_id)
            predictions.append((f"{im_id}_grapheme_root",g[i]))
            predictions.append((f"{im_id}_vowel_diacritic",v[i]))
            predictions.append((f"{im_id}_consonant_diacritic",c[i]))

SyntaxError: invalid syntax (<ipython-input-8-788b755c59ca>, line 15)

In [ ]:
sub = pd.DataFrame(predictions,columns=["row_id","target"])

In [ ]:
sub.to_csv("submission.csv",index=False)